In [1]:
import pandas as pd
import numpy as np
import math 

In [2]:
def multiplication_factor(p_unit, t_unit):
    if p_unit=='KW' and t_unit=='Daily':
        factor = 2725/(24)
    elif p_unit=='KW' and t_unit=='Weekly':
        factor = 2725/(7*24)   
    elif p_unit=='KW' and t_unit=='Monthly':
        factor = 2725/(30*24)
    elif p_unit=='KW' and t_unit=='Yearly':
        factor = 2725/(365*24)
    elif p_unit=='MW' and t_unit=='Daily':
        factor = 2725/(1000*24)
    elif p_unit=='MW' and t_unit=='Weekly':
        factor = 2725/(1000*7*24)   
    elif p_unit=='MW' and t_unit=='Monthly':
        factor = 2725/(1000*30*24)
    else:
        factor = 2725/(1000*365*24)
    
    return factor

# formula for release
def release(P,RL, TWL, eff, power_unit, time_unit):
    mult_factor = multiplication_factor(power_unit,time_unit)
    return P/(mult_factor*(RL - TWL )* eff)


In [3]:
reservoir_capacity = 7000
minm_power = 75 # in MW
TWL = 97 # m
efficiency = 0.8

In [4]:
def interpolator(value,df):
    
#     df = df.sort('Capacity').reset_index(drop=True)
    
   # Boundary cases
    if value <= df.at[0, 'Capacity']:
        return df.at[0, 'Elevation'], df.at[0, 'Area']
    
    if value >= df.at[df.index[-1], 'Capacity']:
        return df.at[df.index[-1], 'Elevation'], df.at[df.index[-1], 'Area']
    
    # Find upper bound index
    upper_idx = None
    for idx in df.index:
        if df.at[idx, 'Capacity'] >= value:
            upper_idx = idx
            break
    
    # upper_idx is guaranteed to exist and be >= 1
    lower_idx = upper_idx - 1
    
    x_low, x_high = df.at[lower_idx, 'Capacity'], df.at[upper_idx, 'Capacity']
    elev_low, elev_high = df.at[lower_idx, 'Elevation'], df.at[upper_idx, 'Elevation']
    area_low, area_high = df.at[lower_idx, 'Area'], df.at[upper_idx, 'Area']
    
    elevation = elev_low + (elev_high - elev_low)/(x_high - x_low) * (value - x_low)
    area = area_low + (area_high - area_low)/(x_high - x_low) * (value - x_low)
    
    return elevation, area

In [5]:
# Reading data from excel sheets
res_df = pd.read_excel("Elevation Storage Area Relationship for reservoir.xlsx", sheet_name='Sheet1', header=0)
inflow_df = pd.read_excel("Inflow and Evapotranspiration data.xlsx", sheet_name='Reservoir_Inflow', header=2)
et_df = pd.read_excel("Inflow and Evapotranspiration data.xlsx", sheet_name='ET', header=2)

In [6]:
# df = inflow_df.iloc[:, 1:20].apply(pd.to_numeric, errors='coerce')

In [7]:
inflow_df['Mean']=inflow_df.iloc[:,1:20].mean(axis=1)
inflow_df

,Days,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,1992,1993,1994,1995,1996,1997,1998,1999,2000,Mean
0,1,73.0,80.5,85.0,92.5,87.7,102.0,95.6,80.6,60.0,...,80.6,70.7,75.2,64.9,77.5,74.0,99.5,79.0,75.9,82.400000
1,2,71.5,79.0,78.7,91.0,88.8,99.3,95.6,79.3,60.0,...,80.6,70.7,75.2,67.0,77.5,72.8,98.0,78.7,75.9,81.494737
2,3,70.0,77.5,76.6,91.0,81.1,98.1,94.2,79.3,62.0,...,79.3,71.1,74.0,67.0,78.7,73.6,96.5,78.0,75.9,80.452632
3,4,71.5,79.0,74.5,89.5,80.1,95.6,92.7,79.3,59.0,...,79.3,71.9,75.2,66.0,77.5,72.8,94.0,78.0,75.5,79.742105
4,5,71.5,98.0,74.5,89.5,83.2,93.4,92.7,77.9,56.0,...,79.3,69.9,75.2,68.1,76.3,72.4,93.1,79.0,74.5,80.273684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,362,85.2,71.4,110.0,75.9,202.0,101.0,81.9,74.0,92.8,...,70.7,78.7,70.5,79.9,76.3,105.0,81.1,79.0,92.3,89.689474
362,363,83.6,73.5,106.0,74.8,138.0,100.0,81.9,70.0,90.2,...,70.7,79.9,69.3,78.7,76.3,104.0,81.1,78.0,92.3,85.242105
363,364,85.2,90.7,100.0,74.8,118.0,98.5,81.9,66.0,88.9,...,70.7,80.3,69.3,78.7,75.5,103.0,80.1,77.3,90.9,86.184211
364,365,80.5,96.4,95.5,73.8,109.0,97.1,81.9,63.0,88.9,...,69.5,78.3,67.0,77.5,75.2,101.0,80.1,76.9,90.9,85.236842


In [8]:
out_df = pd.DataFrame({'Inflow':[0], 'Start Storage':[6386.536081], 'Avg Storage':[0], 'Elevation':[0], 'Area':[0], 'ET':[0], 'ET Loss':[0], 'Release':[0], 'Spill':[0], 'End Storage':[0], 'Avg Storage New':[0],'Spill Power':[0]})

In [10]:
inflow = inflow_df['Mean']*(30*60*24)/(10**6)
ET_mm = et_df['Eva. mm']

In [11]:
for i in range(366):
    out_df.loc[i,'Inflow'] = inflow[i]
    out_df.loc[i,'Avg Storage'] = out_df.loc[i,'Start Storage']
    out_df.loc[i,'Elevation'], out_df.loc[i,'Area'] = interpolator(out_df.loc[i,'Avg Storage'], res_df)
    out_df.loc[i,'ET'] = ET_mm[i]
    out_df.loc[i,'ET Loss'] = out_df.loc[i,'Area'] * out_df.loc[i, 'ET']/1000
    out_df.loc[i,'Release'] = release(minm_power, out_df.loc[i,'Elevation'], TWL, efficiency, 'MW', 'Daily')
    if (out_df.loc[i,'Start Storage'] + out_df.loc[i,'Inflow'] - out_df.loc[i,'ET Loss']-out_df.loc[i,'Release']) > reservoir_capacity:
        out_df.loc[i,'Spill'] = out_df.loc[i,'Start Storage'] + out_df.loc[i,'Inflow']-out_df.loc[i,'ET Loss'] - out_df.loc[i,'Release'] - reservoir_capacity
    else:
        out_df.loc[i,'Spill'] = 0
    out_df.loc[i,'End Storage'] = out_df.loc[i,'Start Storage'] + out_df.loc[i,'Inflow'] - out_df.loc[i,'ET Loss'] - out_df.loc[i,'Release']-out_df.loc[i,'Spill']
    out_df.loc[i,'Avg Storage New'] = (out_df.loc[i,'Start Storage'] + out_df.loc[i,'End Storage'])/2
    print("_ "*0)
    print(f"The calculation for index {i} is {out_df.loc[i,:]}")
    diff = abs(out_df.loc[i,'Avg Storage']-out_df.loc[i,'Avg Storage New'])/out_df.loc[i,'Avg Storage']*100
    while diff >= 0.1:
        out_df.loc[i,'Inflow'] = inflow[i]
        out_df.loc[i,'Avg Storage'] = out_df.loc[i,'Avg Storage New']
        out_df.loc[i,'Elevation'], out_df.loc[i,'Area'] = interpolator(out_df.loc[i,'Avg Storage'], res_df)
        out_df.loc[i,'ET'] = ET_mm[i]
        out_df.loc[i,'ET Loss'] = out_df.loc[i,'Area'] * out_df.loc[i, 'ET']/1000
        out_df.loc[i,'Release'] = release(minm_power, out_df.loc[i,'Elevation'], TWL, efficiency, 'MW', 'Daily')
        if (out_df.loc[i,'Start Storage'] + out_df.loc[i,'Inflow'] - out_df.loc[i,'ET Loss']-out_df.loc[i,'Release']) > reservoir_capacity:
            out_df.loc[i,'Spill'] = out_df.loc[i,'Start Storage'] + out_df.loc[i,'Inflow']-out_df.loc[i,'ET Loss'] - out_df.loc[i,'Release'] - reservoir_capacity
        else:
            out_df.loc[i,'Spill'] = 0
        out_df.loc[i,'End Storage'] = out_df.loc[i,'Start Storage'] + out_df.loc[i,'Inflow'] - out_df.loc[i,'ET Loss'] - out_df.loc[i,'Release']-out_df.loc[i,'Spill']
        out_df.loc[i,'Avg Storage New'] = (out_df.loc[i,'Start Storage'] + out_df.loc[i,'End Storage'])/2
        diff = abs(out_df.loc[i,'Avg Storage']-out_df.loc[i,'Avg Storage New'])/out_df.loc[i,'Avg Storage']*100
    out_df.loc[i+1,'Start Storage'] = out_df.loc[i,'End Storage']
    out_df.loc[i,'Spill Power']=9.81*out_df.loc[i,'Spill']*(out_df.loc[i,'Elevation']-TWL)*efficiency/(1000)


The calculation for index 0 is Inflow                3.559680
Start Storage      6386.536081
Avg Storage        6386.536081
Elevation           377.000000
Area                124.532010
ET                    6.000000
ET Loss               0.747192
Release               2.948886
Spill                 0.000000
End Storage        6386.399683
Avg Storage New    6386.467882
Spill Power           0.000000
Name: 0, dtype: float64

The calculation for index 1 is Inflow                3.520573
Start Storage      6386.399683
Avg Storage        6386.399683
Elevation           376.998708
Area                124.529671
ET                    6.100000
ET Loss               0.759631
Release               2.948900
Spill                 0.000000
End Storage        6386.211725
Avg Storage New    6386.305704
Spill Power                NaN
Name: 1, dtype: float64

The calculation for index 2 is Inflow                3.475554
Start Storage      6386.211725
Avg Storage        6386.211725
Elevation          

In [12]:
out_df

,Inflow,Start Storage,Avg Storage,Elevation,Area,ET,ET Loss,Release,Spill,End Storage,Avg Storage New,Spill Power
0,3.559680,6386.536081,6386.536081,377.000000,124.532010,6.0,0.747192,2.948886,0.0,6386.399683,6386.467882,0.0
1,3.520573,6386.399683,6386.399683,376.998708,124.529671,6.1,0.759631,2.948900,0.0,6386.211725,6386.305704,0.0
2,3.475554,6386.211725,6386.211725,376.996927,124.526447,6.0,0.747159,2.948918,0.0,6385.991202,6386.101463,0.0
3,3.444859,6385.991202,6385.991202,376.994838,124.522665,6.0,0.747136,2.948940,0.0,6385.739984,6385.865593,0.0
4,3.467823,6385.739984,6385.739984,376.992458,124.518356,5.0,0.622592,2.948965,0.0,6385.636250,6385.688117,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
362,3.682459,6999.056090,6999.056090,396.969227,170.631856,6.0,1.023791,2.752576,0.0,6998.962182,6999.009136,0.0
363,3.723158,6998.962182,6998.962182,396.966165,170.624788,6.0,1.023749,2.752604,0.0,6998.908987,6998.935585,0.0
364,3.682232,6998.908987,6998.908987,396.964431,170.620785,6.0,1.023725,2.752620,0.0,6998.814874,6998.861931,0.0
365,3.626526,6998.814874,6998.814874,396.961363,170.613701,6.0,1.023682,2.752648,0.0,6998.665070,6998.739972,0.0
